### 机器学习拟合Openrank指标

Github: https://github.com/BhJia/Openrank-fitting  
数据、使用方法教程及详细分析均在仓库中

In [ ]:
# 引入相关包
import numpy as np
import pandas as pd
import os
import json
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error,explained_variance_score,r2_score,mean_absolute_error
np.set_printoptions(suppress=True)

定义指标分类

In [ ]:
metrics_unprocessed=["active_dates_and_times",
                     "issue_response_time",
                     "issue_resolution_duration",
                     "issue_age",
                     "change_request_response_time",
                     "change_request_resolution_duration",
                     "change_request_age"
                     ]

metrics_processed=["openrank",
                   "technical_fork",
                   "new_contributors",
                   "inactive_contributors",
                   "bus_factor",
                   "issues_new",
                   "issues_closed",
                   "code_change_lines_add",
                   "code_change_lines_remove",
                   "code_change_lines_sum",
                   "change_requests",
                   "change_requests_accepted",
                   "change_requests_reviews"
                  ]

定义数据预处理函数

In [ ]:
# 输入数据路径
data_path="./data/top_300_metrics/"

# 根据指标分类获取指标数据
def get_value(file,metric):
    with open(file) as f:
        data = json.load(f)

        # 如果指标是有统计信息的,取均值
        if metric in metrics_unprocessed:
            data=data['avg']
        if "2021-10-raw" in data:
            data.pop("2021-10-raw")
    f.close()
    return data

# 对时间进行编码
def time_encoding(df):
    df["time"] = pd.to_datetime(df["time"], format="%Y-%m")
    df["Time"] = (df["time"].dt.year - df["time"].dt.year.min()) * 12 + df["time"].dt.month
    return df

# 判断是否是CHAOSS指标
def is_CHAOSS_metric(metric):
    return (metric in metrics_unprocessed) or (metric in metrics_processed)

# 初始化数据表
def init_metric_table(project_path):
    time=[]
    active_dates_and_times=[]
    
    # 先选择active_dates_and_times这一个指标初始化数据表,之后再将其他指标数据加入表中
    with open(os.path.join(project_path,metrics_unprocessed[0]+".json")) as f:
        data=json.load(f)
        for key,value in data.items():
            time.append(key)
            active_dates_and_times.append(np.average(np.array(value)))
    metric_table=pd.DataFrame({"time":time,"active_dates_and_times":active_dates_and_times})
    return metric_table

# 获取整个数据表
def get_project_metric_table(project_path):
    metrics_table=init_metric_table(project_path)
    for file in os.listdir(project_path):
        file_path=os.path.join(project_path,file)
        metric=file[:-5]
        if metric==metrics_unprocessed[0] or not is_CHAOSS_metric(metric):
            continue

        # 获取指标数据
        data=get_value(file_path,metric)

        # 按照时间顺序排列,并填充缺失值
        metrics_table[metric]=metrics_table["time"].map(data).fillna(0)
    return metrics_table

读取数据并进行数据预处理

In [ ]:
metric_table=[]     # 数据表

# 对每个组织
for organization in tqdm(os.listdir(data_path)):
    org_path=os.path.join(data_path,organization)

    # 对组织的每个项目
    for project in os.listdir(org_path):
        project_path=os.path.join(org_path,project)
        proj_metric_table=get_project_metric_table(project_path)
        metric_table.append(proj_metric_table)
        # corr_proj=proj_metric_table.corr()

进一步处理并划分训练集验证集

In [ ]:
# 合并所有项目的数据
data=pd.concat(metric_table).fillna(0)

# 重新设置索引
data = data.reset_index(drop=True)

# 将时间编码
data=time_encoding(data)
Data=data.drop(columns=["time"])

# 划分训练集验证集
test_data=data[data['time']>="2023-01"]
train_data=data[data['time']<"2023-01"]
X_train=train_data.drop(columns=["openrank","time"])
y_train=train_data["openrank"]
X_test=test_data.drop(columns=["openrank","time"])
y_test=test_data["openrank"]

print("Data Shape:{}, Train data shape:{}, Test data Shape:{}".format(Data.shape,X_train.shape,X_test.shape))

查看数据所有特征

In [ ]:
Data.columns

查看时间编码

In [ ]:
Data['Time']

定义一些绘图函数

In [ ]:
# 分布图
def displot(x):
    fig = plt.figure(dpi=150)
    sns.distplot(x)

# 另一种分布图
def hisplot(x):
    fig = plt.figure(dpi=150)
    sns.histplot(x)

# 相关性热力图
def heatmap(X):
    f,ax = plt.subplots(figsize=(15, 15))
    sns.heatmap(X.corr(), annot=True, linecolor='white',linewidths=0.1,cmap="RdBu", fmt= '.1f',ax=ax)

# 直方分布图
def hist(Data):
    Data.hist(bins=30, figsize=(20,15),color='#A50021')

# 双变量的相关性图
def jointplot(x,y,data):
    fig = plt.figure(dpi=150)
    sns.jointplot(x=x, y=y, data=data, kind="hex",color="#A50021",ratio=8, space=0, height=8, marginal_kws={'bins':10,'kde':True})
    plt.xlabel(x, fontsize=15)
    plt.ylabel(y, fontsize=15)
    plt.show()

待拟合变量OpenRank值的分布情况

In [ ]:
Data['openrank'].describe()

两种分布图

In [ ]:
displot(Data['openrank'])

In [ ]:
hisplot(Data['openrank'])

所有特征的分布直方图

In [ ]:
hist(Data)

相关性热力图

In [ ]:
heatmap(Data)

openrank和其他变量的相关性表（降序）

In [ ]:
Data.corr()["openrank"].sort_values(ascending=False)

输入变量与目标变量之间的关系
1. 时序数据和OpenRank值的关系

In [ ]:
jointplot("Time","openrank",Data.iloc[::10])

In [ ]:
jointplot("issue_response_time","openrank",Data[::20])

多重共线性判断

In [ ]:
# 计算X'X特征值
eigenvalues=np.linalg.eigvals(Data.T @ Data)

# 计算条件数
Condition_Number = np.sqrt(np.abs(np.max(eigenvalues)/np.min(eigenvalues)))

print("eigenvalues:{}".format(eigenvalues))
print("Condition_Number:{}".format(Condition_Number))

滞后特征

In [ ]:
# 添加一阶滞后特征
def add_lag_feature(X,lag):
    lag=1
    for column in X.columns:
        X[column + "_Lag"] = X[column].shift(lag).fillna(0)
    return X

X_train=add_lag_feature(X_train,lag=1)
X_test=add_lag_feature(X_test,lag=1)

查看issues_new和其滞后特征issues_new_Lag

In [ ]:
X_train[["issues_new", "issues_new_Lag"]]

定义XGBoost模型与训练验证过程

In [ ]:
# 定义XGBoost回归模型
def train_xgboost(X_train,X_test,y_train,y_test,epoch):

    # 模型建立
    model = xgb.XGBRegressor()

    # 参数设置
    params = {'colsample_bytree': 1,
              'colsample_bylevel': 1,
              'learning_rate': 0.06,
              'max_depth': 9, 
              'alpha': 10,
              'subsample':1,
              'min_child_weight':4,
              'gamma':0.2,
              'reg_alpha':0.1,
              'reg_lambda':0.3,
              'scale_pos_weight':1}

    # 交叉验证
    dtrain = xgb.DMatrix(X_train, label=y_train)
    cv_results = xgb.cv(params, dtrain, num_boost_round=epoch, nfold=5, metrics='rmse', early_stopping_rounds=10)

    # 获取最佳迭代轮数,训练模型
    best_num_boost_rounds = cv_results.shape[0]  # 最佳的迭代轮数
    best_model = xgb.train(params, dtrain, num_boost_round=best_num_boost_rounds)

    # 将测试集数据转换为DMatrix格式
    dtest = xgb.DMatrix(X_test)

    # 使用最佳模型进行预测
    y_pred = best_model.predict(dtest)

    # 评估
    mse = mean_squared_error(y_test, y_pred)
    rmse=np.sqrt(mse)
    score=explained_variance_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # 打印评估结果
    print("RMSE:{}".format(rmse))
    print("MAE:{}".format(mae))
    print("explained_variance_score:{}".format(score))
    print("R2 score:{}".format(r2))

    return best_model,cv_results

进行训练与测试

In [ ]:
best_model_xgb,cv_results=train_xgboost(X_train,X_test,y_train,y_test,epoch=1000)

In [ ]:
print(cv_results)

绘制特征重要性图

In [ ]:
# 获取特征重要性
plt.figure(dpi=200)
plt.rcParams['figure.dpi'] = 200
fig, ax = plt.subplots(figsize=(10, 8))

# 绘制特征重要性
xgb.plot_importance(best_model_xgb, ax=ax)
plt.show()

In [ ]:
# 降序查看特征重要性
xgb_feature_importance=best_model_xgb.get_score(importance_type='weight')
sorted_importance = {feature: importance for feature, importance in sorted(xgb_feature_importance.items(), key=lambda x: x[1], reverse=True)}
for feature, importance in sorted_importance.items():
    print(f'{feature}: {importance}')

SHAP机器学习可解释性

In [ ]:
# 导入并初始化shap
import shap
shap.initjs()

# shap解释器
explainer_xgb = shap.TreeExplainer(best_model_xgb)

# shap value
shap_values_xgb = explainer_xgb(X_train)

绘制两种summary_plot

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))
shap.summary_plot(shap_values_xgb, X_train, plot_type="bar")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))
shap.summary_plot(shap_values_xgb, X_train)

绘制单个样本每个特征的SHAP value贡献度(force_plot和waterfall)

In [ ]:
shap.force_plot(explainer_xgb.expected_value, shap_values_xgb.values[20000])

In [ ]:
shap.plots.waterfall(shap_values_xgb[20000])

#### 降维：特征选择

In [ ]:
# 要剔除的特征
remove_columns=["Time_Lag",
                "inactive_contributors_Lag",
                "new_contributors_Lag",
                "change_requests_accepted_Lag",
                "issue_age_Lag",
                "new_contributors",
                "change_requests_accepted",
                "code_change_lines_add_Lag",
                "code_change_lines_remove",
                "code_change_lines_remove_Lag",
                "code_change_lines_sum_Lag",
                "code_change_lines_sum",
                "change_request_resolution_duration_Lag"
                ]

X_train_reduce=X_train.drop(columns=remove_columns)
X_test_reduce=X_test.drop(columns=remove_columns)

best_model=train_xgboost(X_train_reduce,X_test_reduce,y_train,y_test,epoch=1000)

In [ ]:
# 要剔除的特征
remove_columns=["Time_Lag",
                "inactive_contributors_Lag",
                "new_contributors_Lag",
                "change_requests_accepted_Lag",
                "issue_age_Lag",
                "new_contributors",
                "change_requests_accepted",
                "code_change_lines_add_Lag",
                "code_change_lines_remove",
                "code_change_lines_remove_Lag",
                "code_change_lines_sum_Lag",
                "code_change_lines_sum",
                "change_request_resolution_duration_Lag",
                "technical_fork"
                ]

X_train_reduce=X_train.drop(columns=remove_columns)
X_test_reduce=X_test.drop(columns=remove_columns)

best_model=train_xgboost(X_train_reduce,X_test_reduce,y_train,y_test,epoch=1000)

根据openrank阈值确定项目阶段

In [ ]:
# 分位数
quantiles = y_train.describe().loc[['25%', '50%', '75%']]

# 四阶段划分
def Stage4(openrank):
    q25,q50,q75=quantiles
    if openrank<=q25:
        stage=1
    elif openrank<=q50:
        stage=2
    elif openrank<=q75:
        stage=3
    else:
        stage=4
    return stage

# 两阶段划分
def Stage2(openrank):
    q25,q50,q75=quantiles
    if openrank<=q50:
        stage=1
    else:
        stage=2

    return stage

# 根据划分函数与阶段数划分数据
def get_train_test_data(data,Stage,stage):
    lag=1

    # 获取划分函数和阶段数对应的数据
    data_stage=data[data["openrank"].apply(Stage) == stage]

    # 划分训练集与验证集
    test_data_stage=data_stage[data_stage['time']>="2023-01"]
    train_data_stage=data_stage[data_stage['time']<"2023-01"]
    X_train_stage=train_data_stage.drop(columns=["openrank","time"]).fillna(0)
    y_train_stage=train_data_stage["openrank"]
    X_test_stage=test_data_stage.drop(columns=["openrank","time"]).fillna(0)
    y_test_stage=test_data_stage["openrank"]

    # 滞后特征
    for column in X_train_stage.columns:
        X_train_stage[column + "_Lag"] = X_train_stage[column].shift(lag).fillna(0)
        X_test_stage[column + "_Lag"] = X_test_stage[column].shift(lag).fillna(0)

    # 降维
    X_train_stage=X_train_stage.drop(columns=remove_columns)
    X_test_stage=X_test_stage.drop(columns=remove_columns)
    return X_train_stage,X_test_stage,y_train_stage,y_test_stage
    

两阶段数据

In [ ]:
X_train1,X_test1,y_train1,y_test1=get_train_test_data(data,Stage2,stage=1)
X_train2,X_test2,y_train2,y_test2=get_train_test_data(data,Stage2,stage=2)

两阶段模型训练验证

In [ ]:
best_model1,cv_results=train_xgboost(X_train1,X_test1,y_train1,y_test1,epoch=1000)

In [ ]:
best_model2,cv_results=train_xgboost(X_train2,X_test2,y_train2,y_test2,epoch=1000)

四阶段数据

In [ ]:
X_train1,X_test1,y_train1,y_test1=get_train_test_data(data,Stage4,stage=1)
X_train2,X_test2,y_train2,y_test2=get_train_test_data(data,Stage4,stage=2)
X_train3,X_test3,y_train3,y_test3=get_train_test_data(data,Stage4,stage=3)
X_train4,X_test4,y_train4,y_test4=get_train_test_data(data,Stage4,stage=4)

四阶段模型训练验证

In [ ]:
best_model1,cv_results=train_xgboost(X_train1,X_test1,y_train1,y_test1,epoch=1000)

In [ ]:
best_model2,cv_results=train_xgboost(X_train2,X_test2,y_train2,y_test2,epoch=1000)

In [ ]:
best_model3,cv_results=train_xgboost(X_train3,X_test3,y_train3,y_test3,epoch=1000)

In [ ]:
best_model4,cv_results=train_xgboost(X_train4,X_test4,y_train4,y_test4,epoch=1000)

定义LightGBM模型与训练验证过程

In [ ]:
def train_lightgbm(X_train, X_test, y_train, y_test,epoch):
    # 定义LightGBM回归模型
    model = lgb.LGBMRegressor(objective='regression')

    # 参数设置
    params = {'colsample_bytree': 1,
            'learning_rate': 0.06,
            'max_depth': 9, 
            'alpha': 10,
            'subsample':1,
            'min_child_weight':4,
            'reg_alpha':0.1,
            'reg_lambda':0.3,
            'scale_pos_weight':1,
            "verbose": -1
            }

    # 创建LightGBM的数据对象
    dtrain = lgb.Dataset(X_train, label=y_train)

    # 交叉验证
    cv_results = lgb.cv(params, dtrain, num_boost_round=epoch, nfold=5, metrics='rmse', stratified=False)

    # 使用最佳迭代轮数训练模型
    best_num_boost_rounds = len(cv_results['valid rmse-mean'])
    best_model = lgb.train(params, dtrain, num_boost_round=best_num_boost_rounds)

    # 使用最佳模型进行预测
    y_pred = best_model.predict(X_test)

    # 评估
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    score = explained_variance_score(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # 打印评估结果
    print("RMSE: {}".format(rmse))
    print("MAE: {}".format(mae))
    print("explained_variance_score: {}".format(score))
    print("R2 score: {}".format(r2))

    return best_model,cv_results


进行训练与测试

In [ ]:
best_model_lgb,cv_results=train_lightgbm(X_train,X_test,y_train,y_test,epoch=1000)

In [ ]:
cv_results

绘制特征重要性图

In [ ]:
# 获取特征重要性
plt.figure(dpi=200)
plt.rcParams['figure.dpi'] = 200
fig, ax = plt.subplots(figsize=(10, 8))

# 绘制特征重要性
lgb.plot_importance(best_model_lgb, ax=ax)
plt.show()

In [ ]:
# 降序查看特征重要性
lgb_feature_importance=best_model_lgb.feature_importance()
lgb_feature_name = best_model_lgb.feature_name()
sorted_importance = sorted(zip(lgb_feature_name, lgb_feature_importance), key=lambda x: x[1], reverse=True)

for name, importance in sorted_importance:
    print(f'{name}: {importance}')

In [ ]:
# 合并特征重要性并取平均
combined_importance = {}
for feature, importance in dict(sorted_importance).items():
    combined_importance[feature] = (importance + xgb_feature_importance.get(feature, 0)) / 2

# 按照特征重要性降序排序
sorted_importance_combined = sorted(combined_importance.items(), key=lambda x: x[1], reverse=True)

# 查看特征重要性
for feature, importance in sorted_importance_combined:
    print(f'{feature}: {importance}')

绘制平均后的特征重要性

In [ ]:
# 提取特征名称和对应的重要性值
feature_names = [feature for feature, _ in sorted_importance_combined]
importance_values = [importance for _, importance in sorted_importance_combined]

# 绘制横向柱状图
plt.figure(figsize=(10, 10))
bars=plt.barh(feature_names, importance_values)
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Feature Importance')
plt.gca().invert_yaxis()  

# 添加数值标注
for i, bar in enumerate(bars):
    plt.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2, f'{importance_values[i]:.2f}', ha='left', va='center')
plt.tight_layout()  
plt.show()

In [ ]:
# 导入并初始化shap
import shap
shap.initjs()

# 设置模型的目标
best_model_lgb.params['objective'] ='reg:linear'

# shap解释器
explainer_lgb = shap.TreeExplainer(best_model_lgb)

# shap value
shap_values_lgb = explainer_lgb(X_train)

绘制两种summary_plot

In [ ]:
fig, ax = plt.subplots(dpi=200)
shap.summary_plot(shap_values_lgb, X_train, plot_type="bar")

In [ ]:
fig, ax = plt.subplots(dpi=200)
shap.summary_plot(shap_values_lgb, X_train)

绘制单个样本每个特征的SHAP value贡献度(force_plot和waterfall)

In [ ]:
shap.force_plot(explainer_lgb.expected_value, shap_values_lgb.values[20000])

In [ ]:
fig, ax = plt.subplots(dpi=200)
shap.plots.waterfall(shap_values_lgb[20000])

降维（特征选择）

In [ ]:
# 要剔除的特征
remove_columns=["Time_Lag",
                "inactive_contributors_Lag",
                "new_contributors_Lag",
                "change_requests_accepted_Lag",
                "issue_age_Lag"
                ]

X_train_reduce=X_train.drop(columns=remove_columns)
X_test_reduce=X_test.drop(columns=remove_columns)

best_model=train_lightgbm(X_train_reduce,X_test_reduce,y_train,y_test,epoch=1000)

In [ ]:
# 要剔除的特征
remove_columns=["Time_Lag",
                "inactive_contributors_Lag",
                "new_contributors_Lag",
                "change_requests_accepted_Lag",
                "issue_age_Lag",
                "new_contributors",
                "change_requests_accepted",
                "code_change_lines_add_Lag",
                "code_change_lines_remove"
                ]

X_train_reduce=X_train.drop(columns=remove_columns)
X_test_reduce=X_test.drop(columns=remove_columns)

best_model=train_lightgbm(X_train_reduce,X_test_reduce,y_train,y_test,epoch=1000)

两阶段数据

In [ ]:
X_train1,X_test1,y_train1,y_test1=get_train_test_data(data,Stage2,stage=1)
X_train2,X_test2,y_train2,y_test2=get_train_test_data(data,Stage2,stage=2)

两阶段模型训练验证

In [ ]:
best_model1,cv_results=train_lightgbm(X_train1,X_test1,y_train1,y_test1,epoch=1000)

In [ ]:
best_model2,cv_results=train_lightgbm(X_train2,X_test2,y_train2,y_test2,epoch=1000)

四阶段数据

In [ ]:
X_train1,X_test1,y_train1,y_test1=get_train_test_data(data,Stage4,stage=1)
X_train2,X_test2,y_train2,y_test2=get_train_test_data(data,Stage4,stage=2)
X_train3,X_test3,y_train3,y_test3=get_train_test_data(data,Stage4,stage=3)
X_train4,X_test4,y_train4,y_test4=get_train_test_data(data,Stage4,stage=4)

四阶段模型训练验证

In [ ]:
best_model1,cv_results=train_lightgbm(X_train1,X_test1,y_train1,y_test1,epoch=1000)

In [ ]:
best_model2,cv_results=train_lightgbm(X_train2,X_test2,y_train2,y_test2,epoch=1000)

In [ ]:
best_model3,cv_results=train_lightgbm(X_train3,X_test3,y_train3,y_test3,epoch=1000)

In [ ]:
best_model4,cv_results=train_lightgbm(X_train4,X_test4,y_train4,y_test4,epoch=1000)